In [99]:
%%writefile index.html
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>AI Design Generator (Browser-only)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <style>
    body { font-family: system-ui, sans-serif; margin: 20px; color: #222; }
    .row { display: flex; gap: 12px; flex-wrap: wrap; margin-bottom: 12px; }
    textarea, input, select, button { padding: 8px; font-size: 14px; }
    textarea { width: 100%; height: 140px; }
    .controls { max-width: 900px; }
    .grid { display: grid; grid-template-columns: repeat(auto-fill, minmax(220px, 1fr)); gap: 12px; }
    .card { border: 1px solid #ddd; border-radius: 8px; padding: 8px; }
    .card img { width: 100%; height: auto; display: block; border-radius: 4px; }
    .hint { color: #666; font-size: 12px; }
    .badge { background:#eee; padding:2px 6px; border-radius: 4px; font-size: 12px; }
    #status { margin: 8px 0; }
  </style>
</head>
<body>
  <h1>AI Design Generator</h1>
  <div class="controls">
    <div class="row">
      <textarea id="prompt" placeholder="Describe your design (e.g., 'Minimalist product hero shot, studio lighting, soft shadows, high detail')"></textarea>
    </div>
    <div class="row">
      <label>Provider:
        <select id="provider">
          <option value="auto" selected>Auto (Horde -> Pollinations)</option>
          <option value="pollinations">Pollinations only</option>
          <option value="stable_horde">Stable Horde only</option>
          <option value="one_each">One from each</option>
        </select>
      </label>
      <label>Count:
        <input id="count" type="number" min="1" max="8" value="3"/>
      </label>
      <label>Aspect ratio:
        <select id="aspect">
          <option>1:1</option>
          <option selected>3:4</option>
          <option>4:3</option>
          <option>9:16</option>
          <option>16:9</option>
        </select>
      </label>
      <label>Seed (optional):
        <input id="seed" type="number" placeholder="e.g. 12345"/>
      </label>
    </div>
    <div class="row">
      <label>Horde API key (optional):
        <input id="hordeKey" type="text" placeholder="leave empty for anonymous"/>
      </label>
      <label>Horde Proxy URL (optional):
        <input id="hordeProxy" type="text" placeholder="https://your-worker.workers.dev"/>
      </label>
      <button id="go">Generate</button>
    </div>
    <div id="status" class="hint"></div>
  </div>

  <div id="results" class="grid"></div>

  <script>
    const el = (id) => document.getElementById(id);

    function aspectToSize(ar) {
      const map = { "1:1": [768,768], "3:4": [768,1024], "4:3": [1024,768], "9:16": [768,1365], "16:9": [1365,768] };
      return map[ar] || [768,1024];
    }

    function pollinationsUrls(prompt, count, width, height, seed) {
      const quoted = encodeURIComponent(prompt);
      const base = `https://image.pollinations.ai/prompt/${quoted}?width=${width}&height=${height}&n=1`;
      const startSeed = Number.isInteger(seed) ? seed : Date.now();
      return Array.from({length: count}, (_, i) => `${base}&seed=${startSeed + i}`);
    }

    async function hordeSubmit(prompt, width, height, seed, apiKey, proxyBase) {
      const payload = {
        prompt,
        params: {
          sampler_name: "k_euler",
          cfg_scale: 7,
          seed: Number.isInteger(seed) ? seed : null,
          steps: 28,
          width: Math.round(width / 64) * 64,
          height: Math.round(height / 64) * 64
        },
        nsfw: false,
        censor_nsfw: true,
        trusted_workers: false,
        slow_workers: true
      };
      const base = proxyBase || "https://stablehorde.net/api/v2";
      const r = await fetch(base + "/generate/async", {
        method: "POST",
        headers: {
          "Content-Type": "application/json",
          "accept": "application/json",
          "apikey": apiKey || "0000000000"
        },
        body: JSON.stringify(payload),
        mode: "cors"
      });
      if (!r.ok) throw new Error("Horde submit failed: " + r.status);
      const j = await r.json();
      if (!j.id) throw new Error("No job id from Horde");
      return j.id;
    }

    async function hordePoll(id, apiKey, proxyBase, timeoutMs = 240000, pollMs = 4000) {
      const start = Date.now();
      const base = proxyBase || "https://stablehorde.net/api/v2";
      while (Date.now() - start < timeoutMs) {
        const r = await fetch(`${base}/generate/status/${id}`, {
          headers: { "accept": "application/json", "apikey": apiKey || "0000000000" },
          mode: "cors",
        });
        if (!r.ok) throw new Error("Horde status failed: " + r.status);
        const j = await r.json();
        if (j.done && j.generations && j.generations.length) return j;
        await new Promise(res => setTimeout(res, pollMs));
      }
      throw new Error("Horde timed out");
    }

    function render(items) {
      const container = el("results");
      container.innerHTML = "";
      items.forEach((it, idx) => {
        const card = document.createElement("div");
        card.className = "card";
        const src = it.url || (it.b64 ? ("data:image/png;base64," + it.b64) : "");
        card.innerHTML = `
          <div class="badge">${it.provider || "unknown"}</div>
          ${src ? `<img src="${src}" alt="img-${idx}"/>` : "<div>No image</div>"}
        `;
        container.appendChild(card);
      });
    }

    async function generate() {
      const prompt = el("prompt").value.trim();
      const count = parseInt(el("count").value || "1", 10);
      const aspect = el("aspect").value;
      const seedRaw = el("seed").value.trim();
      const seed = seedRaw ? parseInt(seedRaw, 10) : null;
      const provider = el("provider").value;
      const hordeKey = el("hordeKey").value.trim() || null;
      const proxy = el("hordeProxy").value.trim() || null;

      if (!prompt) {
        el("status").textContent = "Please enter a prompt.";
        return;
      }
      const [w, h] = aspectToSize(aspect);
      el("status").textContent = "Working...";
      el("go").disabled = true;

      try {
        let items = [];
        if (provider === "pollinations") {
          items = pollinationsUrls(prompt, count, w, h, seed).map(u => ({ provider: "pollinations", url: u }));
        } else if (provider === "stable_horde") {
          for (let i = 0; i < count; i++) {
            const id = await hordeSubmit(prompt, w, h, Number.isInteger(seed) ? seed + i : null, hordeKey, proxy);
            const status = await hordePoll(id, hordeKey, proxy);
            const gen = status.generations.find(g => g.img);
            if (gen && gen.img) items.push({ provider: "stable_horde", b64: gen.img });
          }
        } else if (provider === "one_each") {
          try {
            const id = await hordeSubmit(prompt, w, h, seed, hordeKey, proxy);
            const status = await hordePoll(id, hordeKey, proxy);
            const gen = status.generations.find(g => g.img);
            if (gen && gen.img) items.push({ provider: "stable_horde", b64: gen.img });
          } catch (e) {
            console.warn("Horde failed:", e);
          }
          items.push({ provider: "pollinations", url: pollinationsUrls(prompt, 1, w, h, seed)[0] });
          while (items.length < count) {
            const s = Number.isInteger(seed) ? seed + items.length : null;
            items.push({ provider: "pollinations", url: pollinationsUrls(prompt, 1, w, h, s)[0] });
          }
        } else {
          // Auto: try Horde then fallback to Pollinations
          try {
            for (let i = 0; i < count; i++) {
              const id = await hordeSubmit(prompt, w, h, Number.isInteger(seed) ? seed + i : null, hordeKey, proxy);
              const status = await hordePoll(id, hordeKey, proxy);
              const gen = status.generations.find(g => g.img);
              if (gen && gen.img) items.push({ provider: "stable_horde", b64: gen.img });
            }
          } catch (e) {
            console.warn("Horde failed, using Pollinations:", e);
          }
          if (items.length < count) {
            const needed = count - items.length;
            const start = Number.isInteger(seed) ? seed + items.length : null;
            const urls = pollinationsUrls(prompt, needed, w, h, start);
            items.push(...urls.map(u => ({ provider: "pollinations", url: u })));
          }
        }

        render(items);
        el("status").textContent = `Done. Returned ${items.length} image(s).`;
      } catch (e) {
        console.error(e);
        el("status").textContent = "Error: " + (e.message || e);
      } finally {
        el("go").disabled = false;
      }
    }

    document.getElementById("go").addEventListener("click", generate);
  </script>
</body>
</html>

Overwriting index.html


In [100]:
from IPython.display import HTML
HTML(open('index.html', 'r').read())

In [101]:
%%writefile login.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Login</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    body {
      background: linear-gradient(120deg, #a1c4fd 0%, #c2e9fb 100%);
      font-family: 'Segoe UI', Arial, sans-serif;
      min-height: 100vh;
      margin: 0;
      display: flex;
      align-items: center;
      justify-content: center;
    }
    .login-container {
      background: #fff;
      padding: 40px 32px 32px 32px;
      border-radius: 12px;
      box-shadow: 0 4px 24px #0002;
      width: 100%;
      max-width: 350px;
      box-sizing: border-box;
    }
    .login-container h2 {
      text-align: center;
      margin-bottom: 28px;
      color: #007bff;
      font-weight: 600;
    }
    .login-container label {
      display: block;
      margin-bottom: 6px;
      color: #333;
      font-size: 15px;
    }
    .login-container input[type="text"],
    .login-container input[type="password"] {
      width: 100%;
      padding: 10px 12px;
      margin-bottom: 18px;
      border: 1px solid #ccc;
      border-radius: 5px;
      font-size: 15px;
      background: #f8faff;
      transition: border 0.2s;
    }
    .login-container input:focus {
      border: 1.5px solid #007bff;
      outline: none;
    }
    .login-container button {
      width: 100%;
      padding: 12px;
      background: #007bff;
      color: #fff;
      border: none;
      border-radius: 5px;
      font-size: 16px;
      font-weight: 600;
      cursor: pointer;
      margin-top: 8px;
      transition: background 0.2s;
    }
    .login-container button:hover {
      background: #0056b3;
    }
    .login-container .error {
      color: #e74c3c;
      text-align: center;
      margin-top: 10px;
      min-height: 22px;
      font-size: 14px;
    }
    .login-container .register-link {
      text-align: center;
      margin-top: 18px;
      font-size: 15px;
    }
    .login-container .register-link a {
      color: #007bff;
      text-decoration: none;
      font-weight: 500;
    }
    .login-container .register-link a:hover {
      text-decoration: underline;
    }
  </style>
</head>
<body>
  <div class="login-container">
    <h2>Sign In</h2>
    <form id="login-form" autocomplete="on">
      <label for="login-email">Email or Username</label>
      <input id="login-email" type="text" placeholder="Enter your email or username" required />
      <label for="login-password">Password</label>
      <input id="login-password" type="password" placeholder="Enter your password" required />
      <button type="submit">Login</button>
      <div id="login-error" class="error"></div>
    </form>
    <div class="register-link">
      Don't have an account? <a href="setup.html">Sign up</a>
    </div>
  </div>
  <script>
  document.getElementById('login-form').onsubmit = async function(e) {
    e.preventDefault();
    const email = document.getElementById('login-email').value;
    const password = document.getElementById('login-password').value;
    const res = await fetch('/api/login', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ email, password })
    });
    const result = await res.json();
    if (result.success) {
      window.location.href = 'index.html';
    } else {
      document.getElementById('login-error').innerText = result.message || 'Login failed';
    }
  };
  </script>
</body>
</html>
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Login</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    body {
      background: linear-gradient(120deg, #a1c4fd 0%, #c2e9fb 100%);
      font-family: 'Segoe UI', Arial, sans-serif;
      min-height: 100vh;
      margin: 0;
      display: flex;
      align-items: center;
      justify-content: center;
    }
    .login-container {
      background: #fff;
      padding: 40px 32px 32px 32px;
      border-radius: 12px;
      box-shadow: 0 4px 24px #0002;
      width: 100%;
      max-width: 350px;
      box-sizing: border-box;
    }
    .login-container h2 {
      text-align: center;
      margin-bottom: 28px;
      color: #007bff;
      font-weight: 600;
    }
    .login-container label {
      display: block;
      margin-bottom: 6px;
      color: #333;
      font-size: 15px;
    }
    .login-container input[type="text"],
    .login-container input[type="password"] {
      width: 100%;
      padding: 10px 12px;
      margin-bottom: 18px;
      border: 1px solid #ccc;
      border-radius: 5px;
      font-size: 15px;
      background: #f8faff;
      transition: border 0.2s;
    }
    .login-container input:focus {
      border: 1.5px solid #007bff;
      outline: none;
    }
    .login-container button {
      width: 100%;
      padding: 12px;
      background: #007bff;
      color: #fff;
      border: none;
      border-radius: 5px;
      font-size: 16px;
      font-weight: 600;
      cursor: pointer;
      margin-top: 8px;
      transition: background 0.2s;
    }
    .login-container button:hover {
      background: #0056b3;
    }
    .login-container .error {
      color: #e74c3c;
      text-align: center;
      margin-top: 10px;
      min-height: 22px;
      font-size: 14px;
    }
    .login-container .register-link {
      text-align: center;
      margin-top: 18px;
      font-size: 15px;
    }
    .login-container .register-link a {
      color: #007bff;
      text-decoration: none;
      font-weight: 500;
    }
    .login-container .register-link a:hover {
      text-decoration: underline;
    }
  </style>
</head>
<body>
  <div class="login-container">
    <h2>Sign In</h2>
    <form id="login-form" autocomplete="on">
      <label for="login-email">Email or Username</label>
      <input id="login-email" type="text" placeholder="Enter your email or username" required />
      <label for="login-password">Password</label>
      <input id="login-password" type="password" placeholder="Enter your password" required />
      <button type="submit">Login</button>
      <div id="login-error" class="error"></div>
    </form>
    <div class="register-link">
      Don't have an account? <a href="setup.html">Sign up</a>
    </div>
  </div>
  <script>
  document.getElementById('login-form').onsubmit = async function(e) {
    e.preventDefault();
    const email = document.getElementById('login-email').value;
    const password = document.getElementById('login-password').value;
    const res = await fetch('/api/login', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ email, password })
    });
    const result = await res.json();
    if (result.success) {
      window.location.href = 'index.html';
    } else {
      document.getElementById('login-error').innerText = result.message || 'Login failed';
    }
  };
  </script>
</body>
</html>
<<<<<<< HEAD
=======

>>>>>>> 9579070c67e715e8377138f421d9d621cd980fbc

Overwriting login.html


In [102]:
%%writefile setup.html
<div class="row">
  <label style="flex:1 1 100%">
    Other details (optional)
    <textarea id="details" placeholder="Role, team, goals, or anything else you'd like us to know"></textarea>
  </label>
</div>

<div class="row">
  <label>
    Default provider:
    <select id="defaultProvider">
      <option value="auto" selected>Auto (Horde -> Pollinations)</option>
      <option value="pollinations">Pollinations only</option>
      <option value="stable_horde">Stable Horde only</option>
      <option value="one_each">One from each</option>
    </select>
  </label>
  <label>
    Default aspect:
    <select id="defaultAspect">
      <option>1:1</option>
      <option selected>3:4</option>
      <option>4:3</option>
      <option>9:16</option>
      <option>16:9</option>
    </select>
  </label>
  <label>
    Horde API key (optional):
    <input id="hordeKey" type="text" placeholder="leave empty for anonymous"/>
  </label>
  <label>
    Horde Proxy URL (optional):
    <input id="hordeProxy" type="text" placeholder="https://your-worker.workers.dev"/>
  </label>
</div>

<div class="row actions">
  <button id="save">Save and continue</button>
  <button id="skip" type="button">Skip for now</button>
  <div id="status" class="hint"></div>
</div>

Overwriting setup.html


In [103]:
from IPython.display import HTML

setup_html = """<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Setup</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <style>
    body { font-family: system-ui, sans-serif; margin: 20px; color: #222; }
    .row { display: flex; gap: 12px; flex-wrap: wrap; margin-bottom: 12px; }
    textarea, input, select, button { padding: 8px; font-size: 14px; }
    textarea { width: 100%; height: 120px; }
    .controls { max-width: 900px; }
    .hint { color: #666; font-size: 12px; }
    .actions { display:flex; gap:12px; align-items:center; }
    #status { margin: 8px 0; }
  </style>
</head>
<body>
  <h1>Welcome — quick setup</h1>
  <p class="hint">We’ll use this info to personalize your experience. You can change it later. Stored locally on this device.</p>

  <div class="controls">
    <div class="row">
      <label style="flex:1 1 260px">
        Name (required)
        <input id="name" type="text" placeholder="e.g., Alex Chen" autocomplete="name" style="width:100%"/>
      </label>
      <label style="flex:1 1 100%">
        Other details (optional)
        <textarea id="details" placeholder="Role, team, goals, or anything else you'd like us to know"></textarea>
      </label>
    </div>

    <div class="row actions">
      <button id="save">Save and continue</button>
      <button id="skip" type="button">Skip for now</button>
      <div id="status" class="hint"></div>
    </div>
  </div>

  <script>
    const el = (id) => document.getElementById(id);
    const PROFILE_KEY = "aidg_profile";

    function getNextUrl() {
      try {
        const sp = new URLSearchParams(location.search);
        return sp.get("next") || "index.html";
      } catch {
        return "index.html";
      }
    }

    function loadProfile() {
      try {
        const raw = localStorage.getItem(PROFILE_KEY);
        return raw ? JSON.parse(raw) : null;
      } catch {
        return null;
      }
    }

    function saveProfile(profile) {
      localStorage.setItem(PROFILE_KEY, JSON.stringify(profile));
    }

    function prefillForm() {
      const p = loadProfile();
      if (!p) return;
      if (p.name) el("name").value = p.name;
      if (p.details) el("details").value = p.details;
    }

    function validate() {
      const name = el("name").value.trim();
      if (!name) return { ok: false, msg: "Please enter your name." };
      return { ok: true };
    }

    function onSave() {
      el("status").textContent = "";
      const v = validate();
      if (!v.ok) {
        el("status").textContent = v.msg;
        el("name").focus();
        return;
      }
      const profile = {
        name: el("name").value.trim(),
        details: el("details").value.trim(),
        updatedAt: new Date().toISOString(),
        createdAt: loadProfile()?.createdAt || new Date().toISOString()
      };
      try {
        saveProfile(profile);
        el("status").textContent = "Saved.";
        setTimeout(() => {
          location.href = getNextUrl();
        }, 250);
      } catch (e) {
        console.error(e);
        el("status").textContent = "Error saving profile. Please check your browser storage settings.";
      }
    }

    function onSkip() {
      location.href = getNextUrl();
    }

    window.addEventListener("DOMContentLoaded", () => {
      prefillForm();
      el("name").focus();
      el("save").addEventListener("click", onSave);
      el("skip").addEventListener("click", onSkip);
    });
  </script>
</body>
</html>"""

# Save a file you can open in the browser too
with open("setup.html", "w", encoding="utf-8") as f:
    f.write(setup_html)

# Render inline in the notebook
HTML(open('setup.html', 'r', encoding='utf-8').read())

In [104]:
%%writefile setup.html
const saveBtn = document.getElementById('save');
  const skipBtn = document.getElementById('skip');
  const status = document.getElementById('status');

  saveBtn.addEventListener('click', () => {
    const payload = {
      details: document.getElementById('details').value.trim(),
      defaultProvider: document.getElementById('defaultProvider').value,
      defaultAspect: document.getElementById('defaultAspect').value,
      hordeKey: document.getElementById('hordeKey').value.trim(),
      hordeProxy: document.getElementById('hordeProxy').value.trim(),
    };

    // Example: save to localStorage (or send with fetch to your backend)
    localStorage.setItem('appSettings', JSON.stringify(payload));
    status.textContent = 'Saved. Continuing...';
    // e.g., navigate
    // window.location.href = '/next';
  });

  skipBtn.addEventListener('click', () => {
    status.textContent = 'Skipped.';
    // e.g., navigate
    // window.location.href = '/next';
  });

Overwriting setup.html


In [105]:
from IPython.display import HTML

account_html = """<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>Account</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <style>
    :root { color-scheme: light dark; }
    body {
      background: linear-gradient(120deg, #a1c4fd 0%, #c2e9fb 100%);
      font-family: 'Segoe UI', Arial, sans-serif;
      min-height: 100vh;
      margin: 0;
      display: flex;
      align-items: center;
      justify-content: center;
      color: #222;
    }
    .account-container {
      background: #fff;
      padding: 32px;
      border-radius: 12px;
      box-shadow: 0 4px 24px #0002;
      width: 100%;
      max-width: 560px;
      box-sizing: border-box;
    }
    h2 {
      margin: 0 0 18px;
      color: #007bff;
      font-weight: 600;
      text-align: center;
    }
    .field {
      margin-bottom: 16px;
    }
    label {
      display: block;
      margin-bottom: 6px;
      color: #333;
      font-size: 15px;
      font-weight: 600;
    }
    input[type="text"], textarea {
      width: 100%;
      padding: 10px 12px;
      border: 1px solid #ccc;
      border-radius: 6px;
      font-size: 15px;
      background: #f8faff;
      transition: border 0.2s;
      box-sizing: border-box;
    }
    textarea { min-height: 110px; resize: vertical; }
    input:focus, textarea:focus {
      border: 1.5px solid #007bff;
      outline: none;
      background: #fff;
    }
    .row { display: flex; gap: 12px; flex-wrap: wrap; }
    .actions { display: flex; gap: 12px; margin-top: 8px; flex-wrap: wrap; }
    button {
      padding: 10px 14px;
      border: none;
      border-radius: 6px;
      font-size: 15px;
      font-weight: 600;
      cursor: pointer;
    }
    .primary { background: #007bff; color: #fff; }
    .primary:hover { background: #0056b3; }
    .secondary { background: #e9ecef; color: #333; }
    .danger { background: #e74c3c; color: #fff; margin-left: auto; }
    .danger:hover { filter: brightness(0.95); }
    .muted { color: #666; font-size: 13px; }
    .status { min-height: 20px; margin-top: 8px; font-size: 14px; }
    .nav { text-align: center; margin-top: 16px; }
    .nav a { color: #007bff; text-decoration: none; font-weight: 500; }
    .nav a:hover { text-decoration: underline; }
    .notice {
      background: #fff7cc; color: #7a5d00; border: 1px solid #ffe680;
      padding: 10px 12px; border-radius: 8px; margin-bottom: 14px; font-size: 14px;
    }
    @media (prefers-color-scheme: dark) {
      body { color: #eee; }
      .account-container { background: #1f2937; box-shadow: 0 4px 24px #0006; }
      label { color: #e5e7eb; }
      input[type="text"], textarea { background: #111827; border-color: #374151; color: #e5e7eb; }
      .secondary { background: #374151; color: #e5e7eb; }
      .notice { background: #3b3a1f; border-color: #5c5a2a; color: #f2f0c2; }
      .nav a { color: #66aaff; }
    }
  </style>
</head>
<body>
  <div class="account-container">
    <h2>Account</h2>

    <div id="no-profile" class="notice" style="display:none">
      No saved account found. You can create one on the setup page.
    </div>

    <div class="field">
      <label for="acc-name">Name</label>
      <input id="acc-name" type="text" placeholder="Your name" autocomplete="name" />
    </div>

    <div class="field">
      <label for="acc-details">Other details</label>
      <textarea id="acc-details" placeholder="Role, team, goals, notes, etc."></textarea>
    </div>

    <div class="muted" id="timestamps"></div>

    <div class="actions">
      <button id="save" class="primary" disabled>Save changes</button>
      <button id="cancel" class="secondary" disabled>Cancel</button>
      <button id="delete" class="danger" title="Remove saved account data">Delete account data</button>
    </div>

    <div id="status" class="status muted"></div>

    <div class="nav">
      <a href="index.html">Back to Home</a> ·
      <a href="setup.html">Setup Page</a>
    </div>
  </div>

  <script>
    const PROFILE_KEY = "aidg_profile";
    const el = (id) => document.getElementById(id);

    function loadProfile() {
      try {
        const raw = localStorage.getItem(PROFILE_KEY);
        return raw ? JSON.parse(raw) : null;
      } catch {
        return null;
      }
    }

    function saveProfile(p) {
      localStorage.setItem(PROFILE_KEY, JSON.stringify(p));
    }

    function fmt(ts) {
      if (!ts) return "";
      try {
        return new Date(ts).toLocaleString();
      } catch { return ts; }
    }

    let savedProfile = null;   // last-saved copy from storage
    let workingProfile = null; // current edits (mirrors form)

    function refreshTimestamps() {
      const created = savedProfile?.createdAt ? "Created: " + fmt(savedProfile.createdAt) : "";
      const updated = savedProfile?.updatedAt ? " • Last updated: " + fmt(savedProfile.updatedAt) : "";
      el("timestamps").textContent = (created || updated) ? (created + updated) : "";
    }

    function fillFormFrom(profile) {
      el("acc-name").value = profile?.name || "";
      el("acc-details").value = profile?.details || "";
    }

    function snapshotFromForm() {
      return {
        name: el("acc-name").value.trim(),
        details: el("acc-details").value.trim(),
        createdAt: savedProfile?.createdAt || new Date().toISOString(),
        updatedAt: new Date().toISOString(),
      };
    }

    function isDirty() {
      const now = {
        name: el("acc-name").value.trim(),
        details: el("acc-details").value.trim(),
      };
      const base = {
        name: (savedProfile?.name || "").trim(),
        details: (savedProfile?.details || "").trim(),
      };
      return now.name !== base.name || now.details !== base.details;
    }

    function updateDirtyState() {
      const dirty = isDirty();
      el("save").disabled = !dirty;
      el("cancel").disabled = !dirty;
      el("status").textContent = dirty ? "You have unsaved changes." : "";
    }

    function handleSave() {
      const name = el("acc-name").value.trim();
      if (!name) {
        el("status").textContent = "Please enter your name.";
        el("acc-name").focus();
        return;
      }
      const next = snapshotFromForm();
      try {
        saveProfile(next);
        savedProfile = JSON.parse(JSON.stringify(next));
        refreshTimestamps();
        updateDirtyState();
        el("status").textContent = "Saved.";
      } catch (e) {
        console.error(e);
        el("status").textContent = "Error saving. Check browser storage settings.";
      }
    }

    function handleCancel() {
      if (!savedProfile) return;
      fillFormFrom(savedProfile);
      updateDirtyState();
      el("status").textContent = "Changes discarded.";
    }

    function handleDelete() {
      if (!confirm("This will remove your saved account data from this browser. Continue?")) return;
      try {
        localStorage.removeItem(PROFILE_KEY);
        savedProfile = null;
        fillFormFrom({ name: "", details: "" });
        refreshTimestamps();
        updateDirtyState();
        el("no-profile").style.display = "";
        el("status").textContent = "Account data cleared.";
      } catch (e) {
        console.error(e);
        el("status").textContent = "Error clearing data.";
      }
    }

    window.addEventListener("DOMContentLoaded", () => {
      savedProfile = loadProfile();
      if (!savedProfile) {
        el("no-profile").style.display = "";
        savedProfile = { name: "", details: "", createdAt: "", updatedAt: "" };
      }
      fillFormFrom(savedProfile);
      refreshTimestamps();
      updateDirtyState();

      el("acc-name").addEventListener("input", updateDirtyState);
      el("acc-details").addEventListener("input", updateDirtyState);
      el("save").addEventListener("click", handleSave);
      el("cancel").addEventListener("click", handleCancel);
      el("delete").addEventListener("click", handleDelete);
    });
  </script>
</body>
</html>"""

# Save a file you can open in the browser too
with open("account.html", "w", encoding="utf-8") as f:
    f.write(account_html)

# Render inline in the notebook
HTML(account_html)

In [106]:
from IPython.display import HTML

template_html = """<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8"/>
  <title>Templates</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <style>
    :root { color-scheme: light dark; }
    body {
      background: linear-gradient(120deg, #a1c4fd 0%, #c2e9fb 100%);
      font-family: 'Segoe UI', Arial, sans-serif;
      min-height: 100vh;
      margin: 0;
      display: flex;
      align-items: center;
      justify-content: center;
      color: #222;
    }
    .container {
      background: #fff;
      padding: 28px 24px;
      border-radius: 12px;
      box-shadow: 0 4px 24px #0002;
      width: 100%;
      max-width: 900px;
      box-sizing: border-box;
    }
    h2 { margin: 0 0 12px; color: #007bff; font-weight: 600; text-align: center; }
    .sub { text-align:center; color:#555; margin-bottom: 16px; }
    .notice {
      background: #fff7cc; color: #7a5d00; border: 1px solid #ffe680;
      padding: 10px 12px; border-radius: 8px; margin-bottom: 14px; font-size: 14px;
    }
    .templates {
      display: grid;
      grid-template-columns: repeat(auto-fill, minmax(220px, 1fr));
      gap: 10px 14px;
      margin-top: 12px;
    }
    .tpl-item {
      background: #f8faff; padding: 10px 12px; border: 1px solid #dfe7ff;
      border-radius: 8px; display:flex; gap:8px; align-items:flex-start;
    }
    .actions { display:flex; gap:12px; flex-wrap:wrap; margin-top: 14px; }
    button {
      padding: 10px 14px; border: none; border-radius: 6px; font-size: 15px; font-weight: 600; cursor: pointer;
    }
    .primary { background: #007bff; color:#fff; }
    .primary:hover { background: #0056b3; }
    .secondary { background: #e9ecef; color:#333; }
    .danger { background: #e74c3c; color:#fff; }
    .muted { color: #666; font-size: 13px; }
    .row { display:flex; gap: 12px; flex-wrap: wrap; align-items: center; }
    .divider { height:1px; background:#e5e7eb; margin: 18px 0; }
    .gen-area { display:none; }
    .imgs { display:grid; grid-template-columns: 1fr 1fr; gap:14px; align-items:start; }
    .card {
      background:#f8faff; border: 1px solid #dfe7ff; border-radius:8px; padding: 8px;
      display:flex; flex-direction:column; gap:8px; align-items:center; justify-content:center;
    }
    .card img { width: 100%; height: auto; border-radius:6px; background:#fff; }
    .like-actions { display:flex; gap:10px; justify-content:center; }
    .right-actions { display:flex; gap:10px; align-items:center; }
    .status { min-height: 20px; font-size: 14px; margin-top: 8px; }
    .remaining { font-size: 13px; color:#555; }
    .linkbar { text-align:center; margin-top: 16px; }
    .linkbar a { color:#007bff; text-decoration:none; font-weight:500; }
    .linkbar a:hover { text-decoration:underline; }
    @media (max-width: 720px) {
      .imgs { grid-template-columns: 1fr; }
    }
    @media (prefers-color-scheme: dark) {
      body { color:#eee; }
      .container { background:#1f2937; box-shadow: 0 4px 24px #0006; }
      .tpl-item { background:#111827; border-color:#374151; }
      .card { background:#111827; border-color:#374151; }
      .secondary { background:#374151; color:#e5e7eb; }
      .notice { background:#3b3a1f; border-color:#5c5a2a; color:#f2f0c2; }
      .linkbar a { color:#66aaff; }
      .divider { background:#374151; }
    }
  </style>
</head>
<body>
  <div class="container">
    <h2>Templates</h2>
    <div id="no-profile" class="notice" style="display:none">
      No saved account found. You can create one on the setup page. You can still pick templates below.
    </div>
    <div class="sub">
      Assigned to: <span id="user-name">User</span>
    </div>

    <div class="muted">Select your industry templates below (checkboxes). Click Generate when exactly one is selected.</div>
    <div id="template-list" class="templates"></div>

    <div class="actions">
      <button id="save-assign" class="secondary" disabled>Save assignments to my account</button>
      <button id="generate" class="primary" disabled>Generate images from selected template</button>
      <span id="save-status" class="muted"></span>
    </div>

    <div class="divider"></div>

    <div id="gen-area" class="gen-area">
      <div class="row">
        <div><strong>Selected template:</strong> <span id="sel-template"></span></div>
        <div class="remaining" id="remaining"></div>
      </div>
      <div class="imgs">
        <div class="card">
          <img id="img1" alt="Generated option 1"/>
          <div class="like-actions">
            <button id="like1" class="secondary">I like this</button>
          </div>
        </div>
        <div class="card">
          <img id="img2" alt="Generated option 2"/>
          <div class="like-actions">
            <button id="like2" class="secondary">I like that</button>
          </div>
        </div>
      </div>
      <div class="row" style="margin-top:12px; justify-content:space-between;">
        <div class="right-actions">
          <button id="both-bad" class="danger">Both bad — try again</button>
          <span class="remaining" id="limit-note"></span>
        </div>
        <div class="status muted" id="gen-status"></div>
      </div>
    </div>

    <div class="linkbar">
      <a href="index.html">Home</a> ·
      <a href="setup.html">Setup</a> ·
      <a href="account.html">Account</a>
    </div>
  </div>

  <script>
    // Profile storage
    const PROFILE_KEY = "aidg_profile";
    function loadProfile() {
      try { const raw = localStorage.getItem(PROFILE_KEY); return raw ? JSON.parse(raw) : null; }
      catch { return null; }
    }
    function saveProfile(p) { localStorage.setItem(PROFILE_KEY, JSON.stringify(p)); }

    // Default templates list
    const DEFAULT_TEMPLATES = [
      "E-commerce Product",
      "Real Estate Listing",
      "Travel Destination",
      "Restaurant Menu",
      "Fitness Program",
      "SaaS Landing Page",
      "Automotive Ad",
      "Healthcare Clinic",
      "Education Course",
      "Finance / Banking"
    ];

    // IndexedDB for record-keeping
    const DB_NAME = "aidg_records";
    const STORE = "images";
    function openDB() {
      return new Promise((resolve, reject) => {
        const req = indexedDB.open(DB_NAME, 1);
        req.onupgradeneeded = (e) => {
          const db = req.result;
          if (!db.objectStoreNames.contains(STORE)) {
            db.createObjectStore(STORE, { keyPath: "id", autoIncrement: true });
          }
        };
        req.onsuccess = () => resolve(req.result);
        req.onerror = () => reject(req.error);
      });
    }
    async function saveImageRecord(record) {
      try {
        const db = await openDB();
        await new Promise((resolve, reject) => {
          const tx = db.transaction(STORE, "readwrite");
          tx.oncomplete = () => resolve();
          tx.onerror = () => reject(tx.error);
          tx.objectStore(STORE).add(record);
        });
      } catch (e) {
        console.error("Failed to save image record:", e);
      }
    }

    // Seeded PRNG helpers for deterministic image generation
    function xmur3(str) {
      let h = 1779033703 ^ str.length;
      for (let i = 0; i < str.length; i++) {
        h = Math.imul(h ^ str.charCodeAt(i), 3432918353);
        h = (h << 13) | (h >>> 19);
      }
      return function() {
        h = Math.imul(h ^ (h >>> 16), 2246822507);
        h = Math.imul(h ^ (h >>> 13), 3266489909);
        h ^= h >>> 16;
        return h >>> 0;
      }
    }
    function mulberry32(a) {
      return function() {
        var t = a += 0x6D2B79F5;
        t = Math.imul(t ^ (t >>> 15), t | 1);
        t ^= t + Math.imul(t ^ (t >>> 7), t | 61);
        return ((t ^ (t >>> 14)) >>> 0) / 4294967296;
      }
    }
    function seededRngFrom(str) {
      const seedFn = xmur3(str);
      return mulberry32(seedFn());
    }

    // Image generation (canvas) based on template + runId + variant
    const CANVAS_W = 512, CANVAS_H = 320;
    function randColor(rng) {
      const h = Math.floor(rng() * 360);
      const s = 55 + Math.floor(rng() * 40); // 55-95
      const l = 45 + Math.floor(rng() * 35); // 45-80
      return `hsl(${h} ${s}% ${l}%)`;
    }
    function generateImageDataURL(template, runId, variant) {
      const rng = seededRngFrom(template + "|" + runId + "|" + variant);
      const c = document.createElement("canvas");
      c.width = CANVAS_W; c.height = CANVAS_H;
      const ctx = c.getContext("2d");

      // Background gradient
      const g = ctx.createLinearGradient(0, 0, CANVAS_W, CANVAS_H);
      g.addColorStop(0, randColor(rng));
      g.addColorStop(1, randColor(rng));
      ctx.fillStyle = g;
      ctx.fillRect(0, 0, CANVAS_W, CANVAS_H);

      // Random shapes
      const nShapes = 8 + Math.floor(rng() * 10);
      for (let i = 0; i < nShapes; i++) {
        const x = Math.floor(rng() * CANVAS_W);
        const y = Math.floor(rng() * CANVAS_H);
        const w = 30 + Math.floor(rng() * 160);
        const h = 20 + Math.floor(rng() * 120);
        ctx.save();
        ctx.globalAlpha = 0.15 + rng() * 0.35;
        ctx.fillStyle = randColor(rng);
        ctx.beginPath();
        if (rng() < 0.5) {
          ctx.roundRect(x, y, w, h, 6 + rng() * 24);
          ctx.fill();
        } else {
          const r = 12 + Math.floor(rng() * 80);
          ctx.arc(x, y, r, 0, Math.PI * 2);
          ctx.fill();
        }
        ctx.restore();
      }

      // Overlay band
      ctx.save();
      ctx.globalAlpha = 0.25;
      ctx.fillStyle = "#000";
      ctx.fillRect(0, CANVAS_H - 80, CANVAS_W, 80);
      ctx.restore();

      // Template text
      ctx.fillStyle = "#fff";
      ctx.font = "bold 22px Segoe UI, Arial, sans-serif";
      ctx.textBaseline = "middle";
      ctx.shadowColor = "rgba(0,0,0,0.35)";
      ctx.shadowBlur = 4;
      const text = template + " — Option " + variant;
      const metrics = ctx.measureText(text);
      const tx = Math.max(14, (CANVAS_W - metrics.width) / 2);
      const ty = CANVAS_H - 40;
      ctx.fillText(text, tx, ty);

      return c.toDataURL("image/png");
    }

    // DOM helpers
    const $ = (id) => document.getElementById(id);

    // State
    let profile = null;
    let assigned = [];        // saved in profile
    let currentChecked = [];  // current UI checked values
    let runCounter = 0;
    let bothBadCount = 0;
    const BOTH_BAD_MAX = 20;
    let lastDataURL1 = null, lastDataURL2 = null;

    function renderTemplateList(allTemplates, assignedSet) {
      const list = $("template-list");
      list.innerHTML = "";
      allTemplates.forEach((name, idx) => {
        const id = "tpl_" + idx;
        const wrap = document.createElement("label");
        wrap.className = "tpl-item";
        const cb = document.createElement("input");
        cb.type = "checkbox";
        cb.value = name;
        cb.id = id;
        cb.checked = assignedSet.has(name);
        cb.addEventListener("change", onCheckboxChange);
        const span = document.createElement("span");
        span.textContent = name;
        wrap.appendChild(cb);
        wrap.appendChild(span);
        list.appendChild(wrap);
      });
      syncCurrentChecked();
    }

    function syncCurrentChecked() {
      const boxes = Array.from(document.querySelectorAll("#template-list input[type=checkbox]"));
      currentChecked = boxes.filter(b => b.checked).map(b => b.value);
      updateButtonsState();
    }

    function onCheckboxChange() {
      syncCurrentChecked();
      const changed = !arrayEqual(assigned, currentChecked);
      $("save-assign").disabled = !changed;
      // Reset generation area when selection changes
      hideGenArea();
      bothBadCount = 0;
      $("gen-status").textContent = "";
    }

    function arrayEqual(a, b) {
      if (a.length !== b.length) return false;
      const as = [...a].sort(), bs = [...b].sort();
      return as.every((v, i) => v === bs[i]);
    }

    function updateButtonsState() {
      // Generate enabled only when exactly one template is selected
      $("generate").disabled = currentChecked.length !== 1;
    }

    function showRemaining() {
      const left = Math.max(0, BOTH_BAD_MAX - bothBadCount);
      $("remaining").textContent = "Regenerations left: " + left;
      $("limit-note").textContent = left === 0 ? "Limit reached. Choose a different template or refresh." : "";
      $("both-bad").disabled = left === 0;
    }

    function showGenArea(template) {
      $("sel-template").textContent = template;
      $("gen-area").style.display = "";
      showRemaining();
    }

    function hideGenArea() {
      $("gen-area").style.display = "none";
      $("img1").src = "";
      $("img2").src = "";
      lastDataURL1 = lastDataURL2 = null;
    }

    async function persistGeneratedPair(template, runId, d1, d2) {
      const ts = new Date().toISOString();
      const name = profile?.name || "Unknown";
      await saveImageRecord({ ts, template, runId, variant: 1, by: name, dataURL: d1 });
      await saveImageRecord({ ts, template, runId, variant: 2, by: name, dataURL: d2 });
    }

    async function generatePair(template) {
      const runId = ++runCounter;
      $("gen-status").textContent = "Generating...";
      // Generate two images deterministically from seed/template
      const d1 = generateImageDataURL(template, runId, 1);
      const d2 = generateImageDataURL(template, runId, 2);
      // Display
      $("img1").src = d1;
      $("img2").src = d2;
      lastDataURL1 = d1;
      lastDataURL2 = d2;
      // Save records
      await persistGeneratedPair(template, runId, d1, d2);
      $("gen-status").textContent = "Generated and saved.";
    }

    function downloadDataURL(dataURL, filename) {
      const a = document.createElement("a");
      a.href = dataURL;
      a.download = filename;
      document.body.appendChild(a);
      a.click();
      a.remove();
    }

    function onLike(which) {
      if (!lastDataURL1 || !lastDataURL2) return;
      const template = currentChecked[0];
      const ts = new Date().toISOString().replace(/[:.]/g, "-");
      const name = which === 1 ? "image-1" : "image-2";
      const d = which === 1 ? lastDataURL1 : lastDataURL2;
      downloadDataURL(d, `${template}__${name}__${ts}.png`);
      $("gen-status").textContent = "Downloaded " + (which === 1 ? "first" : "second") + " image.";
    }

    async function onBothBad() {
      if (bothBadCount >= BOTH_BAD_MAX) return;
      bothBadCount += 1;
      showRemaining();
      $("gen-status").textContent = "Regenerating...";
      const template = currentChecked[0];
      await generatePair(template);
      // Scroll to top to emphasize templates section (optional behavior)
      // window.scrollTo({ top: 0, behavior: "smooth" });
    }

    function onSaveAssignments() {
      const next = profile ? JSON.parse(JSON.stringify(profile)) : {};
      next.name = next.name || $("user-name").textContent || "User";
      next.details = next.details || "";
      next.assignedTemplates = currentChecked.slice();
      next.createdAt = next.createdAt || new Date().toISOString();
      next.updatedAt = new Date().toISOString();
      try {
        saveProfile(next);
        profile = next;
        assigned = currentChecked.slice();
        $("save-assign").disabled = true;
        $("save-status").textContent = "Assignments saved.";
        setTimeout(() => $("save-status").textContent = "", 1200);
      } catch (e) {
        console.error(e);
        $("save-status").textContent = "Failed to save assignments.";
      }
    }

    async function onGenerate() {
      if (currentChecked.length !== 1) return;
      bothBadCount = 0;
      const template = currentChecked[0];
      showGenArea(template);
      await generatePair(template);
    }

    function init() {
      profile = loadProfile();
      if (!profile) {
        $("no-profile").style.display = "";
      }
      $("user-name").textContent = profile?.name || "User";
      assigned = Array.isArray(profile?.assignedTemplates) ? profile.assignedTemplates.slice() : [];

      // Build the list (union of profile-assigned and defaults to preserve any custom)
      const all = Array.from(new Set([ ...DEFAULT_TEMPLATES, ...assigned ]));
      renderTemplateList(all, new Set(assigned));

      $("save-assign").addEventListener("click", onSaveAssignments);
      $("generate").addEventListener("click", onGenerate);
      $("like1").addEventListener("click", () => onLike(1));
      $("like2").addEventListener("click", () => onLike(2));
      $("both-bad").addEventListener("click", onBothBad);

      // Initial state
      $("save-assign").disabled = true;
      $("generate").disabled = currentChecked.length !== 1;
      showRemaining();
    }

    if (document.readyState === "loading") {
      window.addEventListener("DOMContentLoaded", init);
    } else {
      init();
    }
  </script>
</body>
</html>"""

# Save the page
with open("template.html", "w", encoding="utf-8") as f:
    f.write(template_html)

# Render inline in the notebook (note: for best results, open template.html in a browser)
HTML(template_html)

In [107]:
from IPython.display import HTML

past_html = """<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8"/>
  <title>Past Records</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <style>
    :root { color-scheme: light dark; }
    body {
      background: linear-gradient(120deg, #a1c4fd 0%, #c2e9fb 100%);
      font-family: 'Segoe UI', Arial, sans-serif;
      min-height: 100vh;
      margin: 0;
      display: flex;
      align-items: center;
      justify-content: center;
      color: #222;
    }
    .container {
      background: #fff;
      padding: 28px 24px;
      border-radius: 12px;
      box-shadow: 0 4px 24px #0002;
      width: 100%;
      max-width: 980px;
      box-sizing: border-box;
    }
    h2 { margin: 0 0 8px; color: #007bff; font-weight: 600; text-align: center; }
    .sub { text-align:center; color:#555; margin-bottom: 12px; }
    .notice {
      background: #fff7cc; color: #7a5d00; border: 1px solid #ffe680;
      padding: 10px 12px; border-radius: 8px; margin-bottom: 14px; font-size: 14px;
    }
    .muted { color: #666; font-size: 13px; }
    .divider { height:1px; background:#e5e7eb; margin: 16px 0; }
    .run {
      background:#f8faff; border: 1px solid #dfe7ff; border-radius:8px; padding: 12px; margin-bottom: 14px;
    }
    .run-head { display:flex; flex-wrap:wrap; gap:8px 16px; align-items:center; justify-content:space-between; }
    .run-meta { font-size:14px; color:#333; display:flex; flex-wrap:wrap; gap:10px 16px; }
    .imgs { display:grid; grid-template-columns: repeat(auto-fill, minmax(220px, 1fr)); gap:12px; margin-top:10px; }
    .card { background:#fff; border: 1px solid #e5e7eb; border-radius:8px; padding: 10px; display:flex; flex-direction:column; gap:8px; }
    .card img { width: 100%; height: auto; border-radius:6px; background:#fff; }
    .actions { display:flex; gap:10px; flex-wrap:wrap; }
    button {
      padding: 8px 12px; border: none; border-radius: 6px; font-size: 14px; font-weight: 600; cursor: pointer;
    }
    .primary { background: #007bff; color:#fff; }
    .primary:hover { background:#0056b3; }
    .secondary { background:#e9ecef; color:#333; }
    .linkbar { text-align:center; margin-top: 14px; }
    .linkbar a { color:#007bff; text-decoration:none; font-weight:500; }
    .linkbar a:hover { text-decoration:underline; }
    @media (prefers-color-scheme: dark) {
      body { color:#eee; }
      .container { background:#1f2937; box-shadow: 0 4px 24px #0006; }
      .run { background:#111827; border-color:#374151; }
      .card { background:#0b0f17; border-color:#374151; }
      .secondary { background:#374151; color:#e5e7eb; }
      .notice { background:#3b3a1f; border-color:#5c5a2a; color:#f2f0c2; }
      .divider { background:#374151; }
      .linkbar a { color:#66aaff; }
      .run-meta { color: #e5e7eb; }
    }
  </style>
</head>
<body>
  <div class="container">
    <h2>Past Records</h2>
    <div class="sub">
      Signed in as: <span id="user-name">User</span>
    </div>
    <div id="no-db" class="notice" style="display:none">
      Your browser does not support IndexedDB, so past records cannot be displayed.
    </div>
    <div id="status" class="muted" style="min-height:18px; margin-bottom: 8px;"></div>
    <div id="runs"></div>
    <div id="empty" class="notice" style="display:none">No past records found.</div>

    <div class="divider"></div>
    <div class="linkbar">
      <a href="index.html">Home</a> ·
      <a href="setup.html">Setup</a> ·
      <a href="account.html">Account</a> ·
      <a href="template.html">Templates</a>
    </div>
  </div>

  <script>
    // Profile (for display only)
    const PROFILE_KEY = "aidg_profile";
    function loadProfile() {
      try { const raw = localStorage.getItem(PROFILE_KEY); return raw ? JSON.parse(raw) : null; }
      catch { return null; }
    }

    // IndexedDB setup (same DB and store as template page)
    const DB_NAME = "aidg_records";
    const STORE = "images";

    function openDB() {
      return new Promise((resolve, reject) => {
        if (!("indexedDB" in window)) {
          reject(new Error("IndexedDB not supported"));
          return;
        }
        const req = indexedDB.open(DB_NAME, 1);
        req.onupgradeneeded = () => {
          const db = req.result;
          if (!db.objectStoreNames.contains(STORE)) {
            db.createObjectStore(STORE, { keyPath: "id", autoIncrement: true });
          }
        };
        req.onsuccess = () => resolve(req.result);
        req.onerror = () => reject(req.error);
      });
    }

    function getAllRecords(db) {
      return new Promise((resolve, reject) => {
        const tx = db.transaction(STORE, "readonly");
        const store = tx.objectStore(STORE);
        const req = store.getAll();
        req.onsuccess = () => resolve(req.result || []);
        req.onerror = () => reject(req.error);
      });
    }

    function deleteRecord(db, id) {
      return new Promise((resolve, reject) => {
        const tx = db.transaction(STORE, "readwrite");
        const store = tx.objectStore(STORE);
        const req = store.delete(id);
        req.onsuccess = () => resolve();
        req.onerror = () => reject(req.error);
      });
    }

    function tsToNum(ts, fallback = 0) {
      const n = Date.parse(ts);
      return Number.isNaN(n) ? fallback : n;
    }

    // Keep only the last N runs (grouped by ts). Returns { keptGroups, removedCount }
    async function pruneToLastNRuns(db, N) {
      const all = await getAllRecords(db);
      if (!all.length) return { keptGroups: [], removedCount: 0 };

      // Group by ts (each generation saved the same ts for both images)
      const groupsMap = new Map();
      for (const rec of all) {
        const key = rec.ts || ("id-" + rec.id);
        if (!groupsMap.has(key)) groupsMap.set(key, []);
        groupsMap.get(key).push(rec);
      }
      // Convert to array with sort key
      const groups = Array.from(groupsMap.entries()).map(([key, arr]) => ({
        key,
        ts: key,
        tsNum: tsToNum(arr[0].ts || key, arr[0].id || 0),
        records: arr.sort((a, b) => (a.variant||0) - (b.variant||0))
      }));
      // Sort newest first
      groups.sort((a, b) => b.tsNum - a.tsNum);

      const kept = groups.slice(0, N);
      const toRemove = groups.slice(N);

      let removedCount = 0;
      for (const g of toRemove) {
        for (const rec of g.records) {
          try { await deleteRecord(db, rec.id); removedCount++; } catch (e) { console.warn("Failed delete", rec.id, e); }
        }
      }
      return { keptGroups: kept, removedCount };
    }

    function $(id) { return document.getElementById(id); }

    function downloadDataURL(dataURL, filename) {
      const a = document.createElement("a");
      a.href = dataURL;
      a.download = filename;
      document.body.appendChild(a);
      a.click();
      a.remove();
    }

    function renderRuns(groups, userName) {
      const container = $("runs");
      container.innerHTML = "";
      if (!groups.length) {
        $("empty").style.display = "";
        return;
      }
      $("empty").style.display = "none";

      for (const g of groups) {
        // Basic metadata from first record
        const first = g.records[0] || {};
        const at = new Date(g.ts || first.ts || Date.now());
        const when = isNaN(at.getTime()) ? (g.ts || first.ts || "") : at.toLocaleString();
        const template = first.template || "Unknown template";
        const runId = (first.runId !== undefined) ? first.runId : "N/A";
        const by = first.by || userName || "User";

        const runDiv = document.createElement("div");
        runDiv.className = "run";

        const head = document.createElement("div");
        head.className = "run-head";

        const meta = document.createElement("div");
        meta.className = "run-meta";
        meta.innerHTML = `
          <div><strong>Generated:</strong> ${when}</div>
          <div><strong>Template:</strong> ${template}</div>
          <div><strong>Run ID:</strong> ${runId}</div>
          <div><strong>By:</strong> ${by}</div>
        `;

        const headActions = document.createElement("div");
        headActions.className = "actions";
        const downloadBothBtn = document.createElement("button");
        downloadBothBtn.className = "secondary";
        downloadBothBtn.textContent = "Download both";
        downloadBothBtn.addEventListener("click", () => {
          for (const rec of g.records) {
            const tsSafe = (g.ts || "").replace(/[:.]/g, "-");
            const name = `${template}__option-${rec.variant || "x"}__${tsSafe || Date.now()}.png`;
            downloadDataURL(rec.dataURL, name);
          }
        });
        headActions.appendChild(downloadBothBtn);

        head.appendChild(meta);
        head.appendChild(headActions);
        runDiv.appendChild(head);

        const grid = document.createElement("div");
        grid.className = "imgs";

        for (const rec of g.records) {
          const card = document.createElement("div");
          card.className = "card";

          const img = document.createElement("img");
          img.src = rec.dataURL;
          img.alt = `Generated image option ${rec.variant || ""}`;

          const actions = document.createElement("div");
          actions.className = "actions";
          const btn = document.createElement("button");
          btn.className = "primary";
          btn.textContent = `Download option ${rec.variant || ""}`;
          btn.addEventListener("click", () => {
            const tsSafe = (g.ts || "").replace(/[:.]/g, "-");
            const name = `${template}__option-${rec.variant || "x"}__${tsSafe || Date.now()}.png`;
            downloadDataURL(rec.dataURL, name);
          });

          actions.appendChild(btn);
          card.appendChild(img);
          card.appendChild(actions);
          grid.appendChild(card);
        }

        runDiv.appendChild(grid);
        container.appendChild(runDiv);
      }
    }

    async function init() {
      // Profile name
      const profile = loadProfile();
      $("user-name").textContent = profile?.name || "User";

      let db;
      try {
        db = await openDB();
      } catch (e) {
        $("no-db").style.display = "";
        $("status").textContent = "IndexedDB is unavailable.";
        return;
      }

      $("status").textContent = "Loading records...";
      try {
        // Keep only last 3 runs
        const { keptGroups, removedCount } = await pruneToLastNRuns(db, 3);
        if (removedCount > 0) {
          $("status").textContent = `Cleaned ${removedCount} old image(s). Showing last 3 runs.`;
        } else {
          $("status").textContent = "Showing last 3 runs (or fewer if not available).";
        }
        renderRuns(keptGroups, profile?.name || "User");
      } catch (e) {
        console.error(e);
        $("status").textContent = "Failed to load records.";
      }
    }

    if (document.readyState === "loading") {
      window.addEventListener("DOMContentLoaded", init);
    } else {
      init();
    }
  </script>
</body>
</html>"""

# Save the page
with open("past.html", "w", encoding="utf-8") as f:
    f.write(past_html)

# Render inline in the notebook (note: for best results, open past.html in a browser)
HTML(past_html)